In [ ]:
import pandas as pd
import requests
import time
from unidecode import unidecode
from urllib.parse import quote
from geopy.geocoders import OpenCage
from dotenv import load_dotenv
import os
from geopy.distance import geodesic
import numpy as np

In [ ]:
# Carregar chave da API
load_dotenv()
OPENCAGE_API_KEY = os.getenv("OPENCAGE_API_KEY")
if not OPENCAGE_API_KEY:
    raise ValueError("A chave OPENCAGE_API_KEY não foi encontrada. Verifique o arquivo .env")

In [ ]:
# Carregar base limpa
base_ML = pd.read_csv('imoveis_ML_belo_horizonte.csv', sep=";")

In [ ]:
# Normalizar colunas 'bairro' e 'endereco'
base_ML['bairro_norm'] = base_ML['bairro'].apply(lambda x: unidecode(x.lower().strip()) if isinstance(x, str) else '')
base_ML['endereco_norm'] = base_ML['endereco'].astype(str).str.lower().apply(unidecode)

In [ ]:
# Expandir abreviações comuns
replacements = {
    'av.': 'avenida',
    'r.': 'rua',
    'dr.': 'doutor',
    'eng.': 'engenheiro',
    'prof.': 'professor'
}
for old, new in replacements.items():
    base_ML['endereco_norm'] = base_ML['endereco_norm'].str.replace(old, new, regex=False)

In [ ]:
# Remover espaços extras
base_ML['endereco_norm'] = base_ML['endereco_norm'].str.strip()

In [ ]:
# 2️⃣ Função para buscar CEP e coordenadas
def get_geo_info_from_address(row, api_key):
    geolocator = OpenCage(api_key=api_key)
    logradouro = row['endereco_norm']
    bairro = row['bairro_norm']
    
    if isinstance(bairro, str) and bairro.lower() != 'none' and bairro.lower() != 'outros':
        query_string = f"{logradouro}, {bairro}, Belo Horizonte, Minas Gerais, Brazil"
    else:
        query_string = f"{logradouro}, Belo Horizonte, Minas Gerais, Brazil"
    
    print(f"Buscando dados geo para: {query_string}...")
    
    try:
        location = geolocator.geocode(query_string, timeout=30)

        
        if location and location.raw:
            latitude = location.latitude
            longitude = location.longitude
            components = location.raw.get('components', {})
            cep = components.get('postcode')
            
            # Se CEP não encontrado, tenta geocodificação reversa
            if not cep:
                reverse_loc = geolocator.reverse((latitude, longitude))
                if reverse_loc and reverse_loc.raw:
                    components_rev = reverse_loc.raw.get('components', {})
                    cep = components_rev.get('postcode')
            
            print(f" -> SUCESSO! Lat: {latitude}, Lon: {longitude}, CEP: {cep}")
            return (latitude, longitude, cep)
        else:
            print(" -> Endereço não encontrado.")
            return (None, None, None)
            
    except Exception as e:
        print(f" -> ERRO na geocodificação: {e}")
        return (None, None, None)


In [ ]:
# 3️⃣ Criar DataFrame com endereços únicos
unique_addresses = base_ML[['endereco_norm', 'bairro_norm']].drop_duplicates().reset_index(drop=True)

resultado_geo = unique_addresses.apply(
    lambda row: pd.Series(get_geo_info_from_address(row, api_key=OPENCAGE_API_KEY)),
    axis=1
)
resultado_geo.columns = ["latitude", "longitude", "CEP"]
unique_addresses = pd.concat([unique_addresses, resultado_geo], axis=1)
unique_addresses.rename(columns={'CEP': 'CEP_opencage'}, inplace=True)

In [ ]:
# Juntar resultados geográficos à base original
base_ML_CEPS = pd.merge(
    base_ML,
    unique_addresses,
    on=['endereco_norm', 'bairro_norm'],
    how='left'
)

In [ ]:
# Relatório final e salvar CSV
ceps_encontrados = base_ML_CEPS['CEP_opencage'].notna().sum()
total_imoveis = len(base_ML_CEPS)
taxa_sucesso = (ceps_encontrados / total_imoveis) * 100

print(f"CEPs encontrados: {ceps_encontrados} de {total_imoveis} imóveis.")
print(f"Taxa de sucesso: {taxa_sucesso:.2f}%")

# Salvar
base_ML_CEPS.to_csv("imoveis_zap_belo_horizonte_CEP.csv", index=False, sep=';', encoding="utf-8-sig")

In [ ]:
base_ML_CEPS = base_ML_CEPS.dropna(subset=['CEP_opencage'])

print(f"Imóveis com CEP: {len(base_ML_CEPS)}")

In [ ]:
# Salvando em arquivo CSV
nome_do_arquivo = 'imoveis_belo_horizonte_CEP.csv'
base_ML_CEPS.to_csv(nome_do_arquivo, index=False, sep=';', encoding='utf-8-sig')

In [ ]:
pontos_de_interesse = {
    "praca_da_liberdade": (-19.9375, -43.9333),          # Praça da Liberdade
    "lagoa_da_pampulha": (-19.8583, -43.9744),          # Lagoa da Pampulha
    "parque_americo_renne": (-19.9208, -43.9333),       # Parque Américo Renné Giannetti
    "parque_das_mangabeiras": (-19.9469, -43.8994),     # Parque das Mangabeiras
    "circuito_da_liberdade": (-19.9375, -43.9333),      # Circuito Cultural Praça da Liberdade
    "mercado_central": (-19.9194, -43.9333),            # Mercado Central de Belo Horizonte
    "igreja_sao_jose": (-19.9208, -43.9333)             # Igreja de São José
}

In [ ]:
# Criar coluna de distância para cada ponto de interesse
for nome, coord_ponto in pontos_de_interesse.items():
    coluna_distancia = f'dist_{nome}_km'
    base_ML_CEPS[coluna_distancia] = base_ML_CEPS.apply(
        lambda row: geodesic((row['latitude'], row['longitude']), coord_ponto).kilometers
        if pd.notna(row['latitude']) and pd.notna(row['longitude'])
        else np.nan,
        axis=1
    )

print("\nFeatures de distância criadas com sucesso!")

In [ ]:
#Salvando em arquivo CSV
nome_do_arquivo = 'imoveis_belo_horizonte_ML_distancias.csv'
base_ML_CEPS.to_csv(nome_do_arquivo, index=False, sep=';', encoding='utf-8-sig')